In [4]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain

In [5]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()
classifier = ClueClassifier()

In [ ]:
puzzle = gridmode.puzzle.iloc[2].lower()

attrs, clues = ppp.proccess(puzzle)

#print(puzzle)

print(attrs.keys())
for k in attrs.keys():
    print(attrs[k])
print()

for c in clues:
    clue, clue_type = classifier.classify(c)

    constr = None
    if clue_type == "IDENTITY":
        constr = IdentityConstrain(attrs, clue)
    if clue_type == "NEXT_TO":
        constr = NextToConstrain(attrs, clue)
    if clue_type == "LEFT":
        constr = LeftConstrain(attrs, clue)
    if clue_type == "RIGHT":
        constr = RightConstrain(attrs, clue)
    if clue_type == "DISTANCE":
        constr = DistanceConstrain(attrs, clue)
    if clue_type == "DIRECT_LEFT":
        constr = DirectLeftConstrain(attrs, clue)
    if clue_type == "DIRECT_RIGHT":
        constr = DirectRightConstrain(attrs, clue)
    if clue_type == "POSITION_ABSOLUTE":
        constr = PositionAbsoluteConstrain(attrs, clue)
    if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
        constr = PositionAbsoluteNegativeConstrain(attrs, clue)

    if constr:
        print(constr.get_info())




dict_keys(['name', 'child', 'music', 'heights', 'house nr.'])
['bob', 'alice', 'peter', 'eric', 'arnold', 'carol']
['fred', 'timothy', 'samantha', 'alice', 'meredith', 'bella']
['pop', 'hip hop', 'classical', 'jazz', 'rock', 'country']
['average', 'very tall', 'tall', 'super tall', 'very short', 'short']
['1', '2', '3', '4', '5', '6']

IdentityConstrain:  the person's child is named alice is bob.
attr1:('alice', 'child')
attr2:('bob', 'name')
attributes:{'name': ['bob', 'alice', 'peter', 'eric', 'arnold', 'carol'], 'child': ['fred', 'timothy', 'samantha', 'alice', 'meredith', 'bella'], 'music': ['pop', 'hip hop', 'classical', 'jazz', 'rock', 'country'], 'heights': ['average', 'very tall', 'tall', 'super tall', 'very short', 'short'], 'house nr.': ['1', '2', '3', '4', '5', '6']}

IdentityConstrain:  alice is the person who is tall.
attr1:('alice', 'name')
attr2:('tall', 'heights')
attributes:{'name': ['bob', 'alice', 'peter', 'eric', 'arnold', 'carol'], 'child': ['fred', 'timothy', 'sam